In [ ]:
import numpy as np
import h5py
a = np.random.random(size=(100,20))
h5f = h5py.File('data.h5', 'w')
h5f.create_dataset('dataset_1', data=a)

In [ ]:
h5f.close()

In [ ]:
h5f = h5py.File('data.h5','r')
b = h5f['dataset_1'][:]
h5f.close()

np.allclose(a,b)

In [ ]:
import numpy as np
import h5py
h5f = h5py.File('data2.h5', 'w')
a = np.random.random(size=(100000,200))
b = [1,2,3,4]
d1=h5f.create_dataset('dataset_1', data=a,compression="gzip", compression_opts=9)
d2=h5f.create_dataset('dataset_2', data=b,compression="gzip", compression_opts=9)
d3=h5f.create_dataset('dataset_3', data=a,compression="lzf")
#h5f.close()

In [ ]:
for name in h5f:
    print(name)

In [ ]:
list(h5f.values())

In [ ]:
list(h5f.keys())

In [ ]:
d1.attrs['temperature']=0

In [ ]:
list(d1.attrs)

In [ ]:
h5f.get('dataset_1')

In [ ]:
h5f.close()

In [ ]:
import pickle
pickle.dump( a, open( "save.p", "wb" ) )

In [ ]:
import numpy as np
import h5py
def guardarHDF5(nombreArchivo, valor, nuevo=False):
    """
    Los dataset son para almacenar objetos grandes (ej: numpy.ndarray, listas)
    Los atributos estan asociados a un dataset dado, y son mas eficientes 
    para objetos mas pequeños (escalares y strings)
    
    r. Read only, the file must exist.
    r+. Read and write, the file must exist.
    w. Create file, overwrite if it exists.
    x. Create file, fail if it exists.
    a. Read and write, create otherwise
    
    >>> guardarHDF5("coso.cupdyle",{'nelo':10, 'nelo2':115}, True)
    >>> guardarHDF5("coso.cupdyle",{'nelo':np.asarray([[1,2,3],[4,5,6]]), 'nelo2':115}, False)
    
    
    >>> h5f = h5py.File("coso.cupdyle", 'r')
    >>> list(h5f.keys())
    >>> list(h5f['atributos'].attrs)
    #h5f['nelo'].value
    #h5f['nelo2'].value
    #h5f['atributos'].attrs["nelo"]

    #h5f['atributos'].value
    
    #h5f['atributos'].attrs['nelo']
    >>> h5f.close()
    """
    if nuevo:
        h5f = h5py.File(nombreArchivo, 'w')
        h5f.create_dataset('atributos', data=0)
    else:
        h5f = h5py.File(nombreArchivo, 'r+')
        for k in valor.keys():
            assert k in h5f.keys() or k in h5f['atributos'].attrs

    for k in valor.keys():
        es_atributo=True
        if isinstance(valor[k],np.ndarray):
            if valor[k].shape[0]>1 or valor[k].shape[1]>1:
                es_atributo=False
        if isinstance(valor[k], list):
            es_atributo=False
            
        if es_atributo:
            h5f['atributos'].attrs[k] = valor[k]
            try:
                # si se guardo como atributo, trato de borrar si hay un dataset
                h5f.__delitem__(k)
            except:
                pass
                #h5f.close()
        else:
            # siempre crea uno nuevo, por lo tanto tener cuidado, no guarda listas
            h5f.create_dataset(str(k), data=valor[k],compression="gzip", compression_opts=9)
            try:
                # si se guardo como dataset, trato de borrar si hay un atributo
                h5f['atributos'].attrs.__delitem__(k)
            except:
                pass
                #h5f.close()
    h5f.close()
    return 0

In [ ]:
guardarHDF5("coso.cupdyle",{'nelo':10, 'nelo2':1}, True)


In [ ]:
h5f = h5py.File("coso.cupdyle", 'r')

In [ ]:
list(h5f.keys())
#h5f['nelo'].value
h5f['nelo2'].value
#h5f['atributos'].attrs["nelo"]

#h5f['atributos'].value
#list(h5f['atributos'].attrs)
#h5f['atributos'].attrs['nelo']


In [ ]:
h5f.close()

In [ ]:
dat=np.asarray([[1,2,3],[4,5,6]])

In [ ]:
type(dat) is np.ndarray

In [ ]:
guardarHDF5("coso.cupdyle",{'nelo':dat, 'nelo2':115}, False)

In [ ]:
guardarHDF5("coso.cupdyle",{'nelo':10, 'nelo2':115}, False)

In [ ]:
def cargarHDF5(nombreArchivo, clave):
    """
    carga la clave almacenada en el archivo del tipo HDF5.
    si clave es None, carga todos los datos y los devuelve como un diccionario, cuidado con la dimension
    
    r. Read only, the file must exist.
    r+. Read and write, the file must exist.
    w. Create file, overwrite if it exists.
    x. Create file, fail if it exists.
    a. Read and write, create otherwise
    
    >>> print(cargarHDF5("coso.cupdyle","nelo2"))
    ... 115
    >>> print(cargarHDF5("coso.cupdyle","nelo2"))
    ...    {'nelo2': 115, 'nelo': array([[1, 2, 3], [4, 5, 6]])}
    """
    import numpy as np
    import h5py
    datos = None
    
    h5f = h5py.File(nombreArchivo, 'r')
    
    if clave is not None:
        if isinstance(clave, list):
            datos = {}
            for k in clave:
                try:
                    # busca primero si es un atributo
                    datos[k] = h5f["atributos"].attrs[k]
                except:
                    try:
                        # busca si es un dataset
                        datos[k] = np.asarray(h5f[k])
                    except:
                        pass
        else:
            try:
                # busca primero si es un atributo
                datos = h5f["atributos"].attrs[clave]
            except:
                try:
                    # busca si es un dataset
                    datos = np.asarray(h5f[clave])
                except:
                    pass
    else: #clave es none, devuelvo todo, ojo
        datos = {}
        for k in h5f.keys():
            if k == 'atributos':
                for k in h5f['atributos'].attrs:
                    datos[k] = h5f['atributos'].attrs[k]
            else:
                datos[k] = np.asarray(h5f[k])        
        
    h5f.close()
    return datos

In [ ]:
print(cargarHDF5("coso.cupdyle","nelo2"))

In [ ]:
h5f.close()

In [ ]:
print(cargarHDF5("coso.cupdyle", None))

In [ ]:
import shelve
def guardarSHELVE(nombreArchivo, valor, nuevo=False):
    """
    alamcena todos los datos en archivos pickle individuales
    se almacenan listas sin problemas y casi cualquier tipo de objeto
    
    >>> guardarSHELVE("cosoS.cupydle", {'nelo':[12]}, nuevo=False)
    
    >>> shelf = shelve.open("cosoS.cupydle", flag='r', writeback=False, protocol=2)
    >>> shelf['nelo']
    ... [11, 12]
    >>> shelf.close()
    """
    if nuevo:
        shelf = shelve.open(nombreArchivo, flag='n', writeback=False, protocol=2)
    else:
        shelf = shelve.open(nombreArchivo, flag='w', writeback=False, protocol=2)
        for k in valor.keys():
            assert k in shelf.keys(), "Clave no encontrada en el archivo"
            
    for key in valor.keys():
        if isinstance(valor[key] ,list):
            if nuevo:
                shelf[key] = valor[key]
            else: # ya hay guardada una lista
                tmp = shelf[key]
                tmp.extend(valor[key])
                shelf[key] = tmp
                del tmp
        else:
            shelf[key] = valor[key]
    shelf.close()
    return 0

In [ ]:
import numpy
guardarSHELVE("cosoS.cupydle", {'nelo':[12], 'coso':numpy.asarray([[1,2],[2,3]])}, nuevo=True)

In [ ]:
shelf = shelve.open("cosoS.cupydle", flag='r', writeback=False, protocol=2)
shelf['nelo']

In [ ]:
shelf.close()

In [ ]:
def cargarSHELVE(nombreArchivo, clave):
    """
    carga la clave almacenada en el archivo del tipo shelve.
    si clave es None, carga todos los datos y los devuelve como un diccionario, cuidado con la dimension
    
    'r' Open existing database for reading only (default)
    'w' Open existing database for reading and writing
    'c' Open database for reading and writing, creating it if it doesn’t exist
    'n' Always create a new, empty database, open for reading and writing
    
    >>> print(cargarHDF5("coso.cupdyle","nelo2"))
    ... 115
    >>> print(cargarHDF5("coso.cupdyle",None))
    ...    {'nelo2': 115, 'nelo': array([[1, 2, 3], [4, 5, 6]])}
    >>> print(cargarHDF5("coso.cupdyle","nelo"))
    ... [12]
    """
    datos = None
    with shelve.open(nombreArchivo, flag='r', writeback=False, protocol=2) as shelf:
        if clave is not None:
            assert clave in shelf.keys(), "clave no almacenada " + str(clave)
            datos = shelf[clave]
        else:
            datos = {key: shelf[key] for key in shelf.keys()}
        shelf.close()
    return datos

In [ ]:
print(cargarSHELVE("cosoS.cupydle", "nelo"))

In [ ]:
# tipos de datos compuestos

import numpy as np
import h5py
# Create the compound datatype.
dtype = np.dtype([("id", np.int32), 
                  ("data",      h5py.special_dtype(vlen=str)),
                      ("Temperature",   np.float),
                      ("Pressure",      np.float)])

In [ ]:
import numpy as np
a=np.asarray([[1,2,3],[4,5,6]])
b=np.asarray([[7,8,9],[10,11,12]])
import h5py
h5f = h5py.File("prueba.cupydle", 'w')

d1 = h5f.create_dataset('dato1', data = 10)

g1 = h5f.create_group('pesos')
g1.create_dataset('pesos1', data = a)
g1.create_dataset('pesos2', data = b)

d1 = h5f.create_dataset('dato2', data = 110)

h5f.close()

In [ ]:
h5f = h5py.File("prueba.cupydle", 'r')
print(list(h5f.keys()))
print(h5f['dato1'].value)
print(list(h5f['pesos'].keys()))
print(type(h5f['pesos']))
print(isinstance(h5f['dato1'], h5py.Group))
h5f.close()

In [4]:
datos = {'tipo':                'mlp',
         'nombre':              "coso",
         'numpy_rng':           "cosa",
         'pesos':               [],
         'pesos_iniciales':     [],
         'bias':                [],
         'bias_iniciales':      [],
         'tasaAprendizaje':     0.0,
         'regularizadorL1':     0.0,
         'regularizadorL2':     0.0,
         'momento':             0.0,
         'epocas':              0,
         'toleranciaError':     0.0,
         'tiempoMaximo':        0,
         'costoTRN':            0,
         'costoVAL':            0,
         'costoTST':            0,
         'activacion':          'sigmoidea'
         }

In [3]:
import numpy as np
import h5py
def guardarHDF5(nombreArchivo, valor, nuevo=False):
    """
    Los dataset son para almacenar objetos grandes (ej: numpy.ndarray, listas)
    Los atributos estan asociados a un dataset dado, y son mas eficientes 
    para objetos mas pequeños (escalares y strings)
    
    r. Read only, the file must exist.
    r+. Read and write, the file must exist.
    w. Create file, overwrite if it exists.
    x. Create file, fail if it exists.
    a. Read and write, create otherwise
    
    >>> guardarHDF5("coso.cupdyle",{'nelo':10, 'nelo2':115}, True)
    >>> guardarHDF5("coso.cupdyle",{'nelo':np.asarray([[1,2,3],[4,5,6]]), 'nelo2':115}, False)
    
    
    >>> h5f = h5py.File("coso.cupdyle", 'r')
    >>> list(h5f.keys())
    >>> list(h5f['atributos'].attrs)
    #h5f['nelo'].value
    #h5f['nelo2'].value
    #h5f['atributos'].attrs["nelo"]

    #h5f['atributos'].value
    
    #h5f['atributos'].attrs['nelo']
    >>> h5f.close()
    """
    if nuevo:
        h5f = h5py.File(nombreArchivo, 'w')
        h5f.create_dataset('atributos', data=0)
    else:
        h5f = h5py.File(nombreArchivo, 'r+')
        for k in valor.keys():
            assert k in h5f.keys() or k in h5f['atributos'].attrs

    for k in valor.keys():
        es_atributo=True
        if isinstance(valor[k],np.ndarray):
            if valor[k].shape[0]>1 or valor[k].shape[1]>1:
                es_atributo=False
        if isinstance(valor[k], list):
            es_atributo=False
            
        if es_atributo:
            h5f['atributos'].attrs[k] = valor[k]
            try:
                # si se guardo como atributo, trato de borrar si hay un dataset
                h5f.__delitem__(k)
            except:
                pass
                #h5f.close()
        else:
            if nuevo:
                if isinstance(valor[k], list):
                    h5f.create_group(k)
                else:
                    h5f.create_dataset(str(k), data=valor[k],compression="gzip", compression_opts=9)
            else:
                if isinstance(h5f[k], h5py.Group):
                    #if isinstance(valor[k], list):
                    # es como una lista, debo determinar el indice e incrementarlo dentro del grupo (lista)
                    dd=h5f[k].items()
                    import collections
                    dd = collections.OrderedDict(sorted(dd))
                    cantidad = len(dd)
                    #print(valor[k])
                    #print(k+str(cantidad))
                    h5f[k].create_dataset(k+str(cantidad),data=valor[k][0])
                else:
                    h5f[k] = valor[k]
            try:
                # si se guardo como dataset, trato de borrar si hay un atributo
                h5f['atributos'].attrs.__delitem__(k)
            except:
                pass
                #h5f.close()
    h5f.close()
    return 0

In [ ]:
import numpy as np
import h5py
h5f = h5py.File("prueba.cupydle", 'r')
for k in h5f.keys():
    print(k,h5f[k])
print(list(h5f['pesos'].keys()))
print(h5f["atributos"].attrs)
print(h5f['pesos']['pesos1'].value)
print(h5f['pesos'])
print([v.value for k,v in h5f['pesos'].items()])
h5f.close()

In [ ]:
import numpy as np
a=np.asarray([[1,2,3],[4,5,6]])
b=np.asarray([[7,8,9],[10,11,12]])
import h5py
h5f = h5py.File("prueba.cupydle", 'w')
for k in datos.keys():
    if isinstance(datos[k], list):
        h5f.create_group(k)
        try:
            h5f.create_group(k)
        except:
            pass
    else:
        h5f.create_dataset(k, data=datos[k])
h5f.close()

In [ ]:
dd={'pesos3':0, 'pesos2':1}
import collections
dd = collections.OrderedDict(sorted(dd.items()))
for k, v in dd.items(): 
    print(k, v)

In [5]:
guardarHDF5("prueba.cupydle", datos, True)

0

In [12]:
import numpy as np
a=np.asarray([[7,8,9],[10,11,12]])
guardarHDF5("prueba.cupydle",{'pesos':a+1}, False)

0

In [ ]:
h5f = h5py.File("prueba.cupydle", 'r')
import collections
dd = h5f.items()
dd = collections.OrderedDict(sorted(dd))
print(dd)
h5f.close()

In [20]:
def cargarHDF5(nombreArchivo, clave):
    """
    carga la clave almacenada en el archivo del tipo HDF5.
    si clave es None, carga todos los datos y los devuelve como un diccionario, cuidado con la dimension
    
    r. Read only, the file must exist.
    r+. Read and write, the file must exist.
    w. Create file, overwrite if it exists.
    x. Create file, fail if it exists.
    a. Read and write, create otherwise
    
    >>> print(cargarHDF5("coso.cupdyle","nelo2"))
    ... 115
    >>> print(cargarHDF5("coso.cupdyle","nelo2"))
    ...    {'nelo2': 115, 'nelo': array([[1, 2, 3], [4, 5, 6]])}
    """
    import numpy as np
    import h5py
    datos = None
    
    h5f = h5py.File(nombreArchivo, 'r')
    
    if clave is not None:
        if isinstance(clave, list):
            datos = {}
            for k in clave:
                # busca primero si es un atributo
                if k in h5f["atributos"].attrs:
                    datos[k] = h5f["atributos"].attrs[k]
                elif isinstance(h5f[k], h5py.Group):    # busca si es un grupo (lista)
                    #datos[k] = []
                    #datos[k] = np.asarray(h5f[k])
                    # guarda una lista con los arrays contenidos en el grupo, ordenado
                    import collections
                    datos[k] = [np.asarray(val.value) for key, val in collections.OrderedDict(sorted(h5f[k].items())).items()]
                else:
                    datos[k] = np.asarray(h5f[k])                                             
        else:
            # busca primero si es un atributo
            if clave in h5f["atributos"].attrs:
                datos = h5f["atributos"].attrs[clave]
            elif isinstance(h5f[clave], h5py.Group):    # busca si es un grupo (lista)
                # guarda una lista con los arrays contenidos en el grupo, ordenado
                import collections
                datos = [np.asarray(val.value) for key, val in collections.OrderedDict(sorted(h5f[clave].items())).items()]
            else:
                datos = np.asarray(h5f[clave])
    else: #clave es none, devuelvo todo, ojo
        datos = {}
        for k in h5f.keys():
            if k == 'atributos':
                for k in h5f['atributos'].attrs:
                    datos[k] = h5f['atributos'].attrs[k]
            elif isinstance(h5f[k], h5py.Group):
                import collections
                datos[k] = [np.asarray(val.value) for key, val in collections.OrderedDict(sorted(h5f[k].items())).items()]
            else:
                datos[k] = np.asarray(h5f[k])        
    h5f.close()
    return datos

In [23]:
#cargarHDF5("prueba.cupydle",['nombre', 'pesos'])
cargarHDF5("prueba.cupydle",None)
cargarHDF5("prueba.cupydle",'pesos')

[array([7, 8, 9]), array([7, 8, 9]), array([ 8,  9, 10])]

In [4]:
guardarHDF5("prueba.cupydle", datos, True)

0

In [10]:
import numpy as np
a=np.asarray([[7,8,9],[10,11,12]])
guardarHDF5("prueba.cupydle",{'pesos':[a]}, False)
guardarHDF5("prueba.cupydle",{'tipo':"mlp2"}, False)


[array([[ 7,  8,  9],
       [10, 11, 12]])]
pesos2


0

In [13]:
cargarHDF5("prueba.cupydle","tipo")

'mlp2'

In [1]:
datos = {'nombre': 'mlp', 'valor': 0.0, 'pesos': []}

In [2]:
datos

{'nombre': 'mlp', 'pesos': [], 'valor': 0.0}